In [14]:
import os
import random
import shutil

data_dir = 'apple_tomatoes'
apples_dir = os.path.join(data_dir, 'apples')
tomatoes_dir = os.path.join(data_dir, 'tomatoes')

#Aprasom kokia dalis tenka mokymui ir testavimui
splits = (0.7, 0.15, 0.15)

def split_data(directory:str, splits:tuple):
    """
    Funkcija yra skirta suskaidyti pateiktam kataloge esancias nuotraukas i tris naujus katalogus, pagal 
    pateiktus isskaidymo dydzius

    Parametrai:
    directory - nuoroda iki failo, kuri norite skaidyti
    splits - tuple, su nurodytais kiekiais mokymui, testavimui ir validacijai
    """
    images = os.listdir(directory)
    random.shuffle(images)
    train_size = int(len(images) * splits[0])
    validation_size = int(len(images) * splits[1])
    print(validation_size)

    train_dir = os.path.join(directory, 'train')
    validation_dir = os.path.join(directory, 'validation')
    test_dir = os.path.join(directory, 'test')

    
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for i, image in enumerate(images):
        if i < train_size:
            shutil.copy(os.path.join(directory, image), os.path.join(train_dir, image))
        elif i < train_size + validation_size:
            shutil.copy(os.path.join(directory, image), os.path.join(validation_dir, image))
        else:
            shutil.copy(os.path.join(directory, image), os.path.join(test_dir, image))


split_data(apples_dir, splits)
split_data(tomatoes_dir, splits)


25


PermissionError: [Errno 13] Permission denied: 'apple_tomatoes\\apples\\validation'

In [ ]:
from PIL import Image

data_dir = 'apple_tomatoes'
apples_dir = os.path.join(data_dir, 'apples')
tomatoes_dir = os.path.join(data_dir, 'tomatoes')

def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError):
        return False
    
def get_valid_image_files(directory):
    """Skirta atrinkti į sąrašą tik validžių failų pavadinimus"""
    valid_files = []
    for root, _, files in os.walk(directory): # naudojame _ katalogams train, validation, test, nes neketiname jų naudoti
        if root != directory:
            for file in files:
                file_path = os.path.join(root, file)
                if is_valid_image(file_path):
                    valid_files.append(file_path)
    return valid_files

valid_cat_photos = get_valid_image_files(apples_dir)
valid_dog_photos = get_valid_image_files(tomatoes_dir)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

dataGen = ImageDataGenerator(rescale=1./255)

apples_df = pd.DataFrame({'filename':valid_cat_photos})
tomatoes_df = pd.DataFrame({'filename':valid_dog_photos})

apples_generator = dataGen.flow_from_dataframe(
    dataframe = apples_df,
    x_col = 'filename',
    target_size = (150, 150),
    batch_size = 20,
    class_mode = None)

tomatoes_generator = dataGen.flow_from_dataframe(
    dataframe = tomatoes_df,
    x_col = 'filename',
    target_size = (150, 150),
    batch_size = 20,
    class_mode = None)

Found 164 validated image filenames.
Found 173 validated image filenames.


In [18]:
apples_generator

In [ ]:
from tensorflow.keras.utils import Sequence
import numpy as np

class CombinedGenerator(Sequence):
    def __init__(self, *generators):
        self.generators = generators
        self._num_batches = sum(len(gen) for gen in generators)
        self.current_generator = 0

    def __len__(self):
        return self._num_batches
    
    def __getitem__(self, idx):
        for gen in self.generators:
            if idx < len(gen):
                batch = gen[idx]
                labels = np.array([0] * batch.shape[0]) if gen == apples_generator else np.array([1]* batch.shape[0])
                return batch, labels
            idx -= len(gen)

combined_generator = CombinedGenerator(apples_generator, tomatoes_generator)

In [65]:
from tensorflow.keras import datasets, layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation = 'relu' ),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\sarun\OneDrive\Documents\Code\Testinei\mokymai\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [67]:
history = model.fit(
    combined_generator,
    steps_per_epoch = len(combined_generator),
    epochs = 7
)

Epoch 1/7
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 157ms/step - accuracy: 0.5428 - loss: 3.2227
Epoch 2/7
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/7


C:\Users\sarun\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.3374 - loss: 0.6994
Epoch 4/7
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 168us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/7
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.7041 - loss: 0.6961
Epoch 6/7
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 108us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/7
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.5516 - loss: 0.6923


In [70]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    if image_path.endswith(".png"):
        im = Image.open(image_path)
        img = im.convert('RGB')
        # rgb_im.save('colors.jpg')

    img = img.resize((150,150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    # print(img_array)
    return img_array

def predict_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    return "pomidoras" if prediction[0][0] > 0.5 else 'obuolys'

print(predict_image('obuolys.png'))
print(predict_image('pomidoras.jpg'))
print(predict_image('obuolys2.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
pomidoras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
pomidoras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
pomidoras
